## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-23-02-32-42 +0000,nyt,Live Updates: Zelensky Criticizes European All...,https://www.nytimes.com/live/2026/01/22/us/tru...
1,2026-01-23-02-25-17 +0000,nyt,U.S. Formally Withdraws From World Health Orga...,https://www.nytimes.com/2026/01/22/us/politics...
2,2026-01-23-02-24-05 +0000,bbc,Teens among the missing after landslide at cam...,https://www.bbc.com/news/articles/c14rr5pnd80o...
3,2026-01-23-02-23-24 +0000,nyt,Mark Carney Says Firmly That ‘Canada Doesn’t L...,https://www.nytimes.com/2026/01/22/world/canad...
4,2026-01-23-02-21-57 +0000,wapo,Judge limits government actions against plaint...,https://www.washingtonpost.com/nation/2026/01/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2411/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,81
203,greenland,22
451,up,20
707,davos,19
19,new,19


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
314,2026-01-22-11-19-40 +0000,nypost,"Trump claims ‘Iran does want to talk, and we’l...",https://nypost.com/2026/01/22/us-news/trump-cl...,152
275,2026-01-22-14-32-36 +0000,nypost,Trump vows US will have ‘total access’ to Gree...,https://nypost.com/2026/01/22/us-news/trump-vo...,152
338,2026-01-22-10-34-23 +0000,nypost,Trump unveils ‘Board of Peace’ in Davos ceremo...,https://nypost.com/2026/01/22/us-news/trump-un...,149
154,2026-01-22-21-19-43 +0000,bbc,"Board of Peace, Zelensky and Musk - What happe...",https://www.bbc.com/news/videos/c78vvvn4qnlo?a...,146
369,2026-01-22-07-39-58 +0000,bbc,Trump says 'framework of a future deal' discus...,https://www.bbc.com/news/articles/cgezx40r7d7o...,143


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
314,152,2026-01-22-11-19-40 +0000,nypost,"Trump claims ‘Iran does want to talk, and we’l...",https://nypost.com/2026/01/22/us-news/trump-cl...
206,72,2026-01-22-18-36-55 +0000,nypost,Elon Musk’s Greenland jokes fall flat at Davos...,https://nypost.com/2026/01/22/business/elon-mu...
72,60,2026-01-22-23-45-00 +0000,wsj,A resolution to rein in President Trump’s powe...,https://www.wsj.com/politics/policy/gop-lawmak...
153,57,2026-01-22-21-22-31 +0000,nyt,"Zelensky Laces Into Europe, Saying It Must Ste...",https://www.nytimes.com/2026/01/22/world/europ...
128,54,2026-01-22-22-03-39 +0000,nypost,Maker of NYC’s hard-to-find trash bins leaves ...,https://nypost.com/2026/01/22/us-news/maker-of...
183,54,2026-01-22-19-43-16 +0000,nyt,Judge Rejects DOJ’s Criminal Complaint Against...,https://www.nytimes.com/2026/01/22/us/politics...
168,49,2026-01-22-20-18-11 +0000,nypost,"JD Vance backs ICE in Minn., urges everyone to...",https://nypost.com/2026/01/22/us-news/jd-vance...
275,44,2026-01-22-14-32-36 +0000,nypost,Trump vows US will have ‘total access’ to Gree...,https://nypost.com/2026/01/22/us-news/trump-vo...
5,40,2026-01-23-02-20-00 +0000,wsj,California Gov. Gavin Newsom spent his days at...,https://www.wsj.com/politics/policy/the-newsom...
130,37,2026-01-22-22-02-00 +0000,wsj,JPMorgan CEO Jamie Dimon Reaps $43 Million in ...,https://www.wsj.com/finance/banking/jpmorgan-c...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
